In [1]:
# install required libraries
# !pip install pandas

In [2]:
# import required libraries
import os
import pprint

import pandas as pd
from sklearn.linear_model import LogisticRegression

In [3]:
# set global variables
DATA_DIR = "../data"

In [4]:
# load the dataset
df = pd.read_csv(os.path.join(DATA_DIR, "train_data.csv"))
df.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
0,450407,Cash loans,F,N,Y,1,67500.0,227520.0,11065.5,180000.0,...,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0,0
1,271298,Cash loans,M,Y,Y,1,247500.0,1882372.5,65560.5,1719000.0,...,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0,0
2,122238,Cash loans,M,Y,Y,1,180000.0,101880.0,10827.0,90000.0,...,0,0,0,0.0,0.0,0.0,2.0,0.0,1.0,0
3,305311,Cash loans,M,N,N,0,81000.0,405000.0,20677.5,405000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,0
4,414121,Cash loans,F,N,Y,0,157500.0,888840.0,29506.5,675000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,0


Above we can see a sample of the dataset. Here for each `SK_ID_CURR` we need to predict the `TARGET` column.

Let's look at data more deeply.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184506 entries, 0 to 184505
Columns: 122 entries, SK_ID_CURR to TARGET
dtypes: float64(65), int64(41), object(16)
memory usage: 171.7+ MB


We can observer from the output of above cell that:  
- There are `1,84,506` data points in total
- There are `122` columns, including the target column
- Columns with different datatypes are present:
    - `65` columns of float datatype,
    - `41` columns of integer datatype, and
    - `16` columns of object datatype

Let us check if there are any duplicate rows in the dataset

In [6]:
duplicated_rows = len(df.index[df.duplicated()])
print(f"There are {duplicated_rows} duplicate rows in the dataset.")

There are 0 duplicate rows in the dataset.


As we can see, there are no duplicate rows in the dataset.  
  
There are two features that have the same value in all rows. We can drop them as they have no relation with the `TARGET` feature.

In [7]:
print(f"Number of columns before: {df.shape[1]}")
df.drop(
    columns=["FLAG_MOBIL", "FLAG_DOCUMENT_12"],
    axis=1,
    inplace=True,
)
print(f"Number of columns after: {df.shape[1]}")

Number of columns before: 122
Number of columns after: 120


Also, we do not need `SK_ID_CURR` columns as this is a unique identifier for each row in the dataset. So let's remove this column.

In [8]:
df.drop(
    columns=["SK_ID_CURR"],
    axis=1,
    inplace=True,
)

## Correlation: 
Let's find correlatation between the `TARGET` columns and rest of the columns (numeric and boolean only) to look at the correlation matrix to find the correlation across various columns in the dataset. 
  
The correlation between two features measures the relationship between the two.
- A correlation value lies between -1 and +1.
- A positive value means if value of one feature increases then value of other feature also increases.
- A negative value means if value of one feature decrease then value of other feature also decreases.
- A neutral value or value close to 0 means there is no (or very little) relation between the two features.

In [9]:
corr_with_target = df.corr(numeric_only=True)["TARGET"]

for column_name, corr_value in corr_with_target.items():
    print("{: <30} -    {:<8}".format(column_name, corr_value))

CNT_CHILDREN                   -    0.020010553326490724
AMT_INCOME_TOTAL               -    -0.0006852676062051972
AMT_CREDIT                     -    -0.030652951616139176
AMT_ANNUITY                    -    -0.011599828617349777
AMT_GOODS_PRICE                -    -0.0400548524029155
REGION_POPULATION_RELATIVE     -    -0.03902845407824006
DAYS_BIRTH                     -    0.08082074831051814
DAYS_EMPLOYED                  -    -0.04687502731897162
DAYS_REGISTRATION              -    0.043995293014956395
DAYS_ID_PUBLISH                -    0.051050601014469205
OWN_CAR_AGE                    -    0.038362821221577914
FLAG_EMP_PHONE                 -    0.04795612917732592
FLAG_WORK_PHONE                -    0.029652406865422486
FLAG_CONT_MOBILE               -    0.0005146966577401499
FLAG_PHONE                     -    -0.02333751593462305
FLAG_EMAIL                     -    -0.001688633457494859
CNT_FAM_MEMBERS                -    0.009743293697398635
REGION_RATING_CLIENT        

Now, based on the value of correlation value we can do the following:
- If the absolute correlation value is high, then that means the two features are highley related to each other. And so we can drop the feature as it contributes very less to the learning.
- If the absolute correlation value is close to zero, then that means the two features have no relationship between them. And so we can drop it too as it contributes very less to the learning.

In [10]:
delete_corr_gt = 0.95
delete_corr_lt = 0.005

num_of_dropped_columns = 0
print("Dropping columns with absolute correlation value:")
for column_name, corr_value in corr_with_target.items():
    if column_name == "TARGET":
        continue
        
    elif (abs(corr_value) > delete_corr_gt) or (abs(corr_value) < delete_corr_lt): 
        num_of_dropped_columns += 1
        print("{: <30} -    {:<8}".format(column_name, abs(corr_value)))
        df.drop(
            columns=[column_name],
            axis=1,
            inplace=True,
        )

print(f"\nTotal number of columns dropped: {num_of_dropped_columns}")
print(f"New shape of dataset: {df.shape}")

Dropping columns with absolute correlation value:
AMT_INCOME_TOTAL               -    0.0006852676062051972
FLAG_CONT_MOBILE               -    0.0005146966577401499
FLAG_EMAIL                     -    0.001688633457494859
REG_REGION_NOT_LIVE_REGION     -    0.004224549029762243
LIVE_REGION_NOT_WORK_REGION    -    0.0025734752842316187
NONLIVINGAPARTMENTS_AVG        -    0.0045460601976467325
NONLIVINGAPARTMENTS_MODE       -    0.002124078503424255
NONLIVINGAPARTMENTS_MEDI       -    0.003907879593400008
FLAG_DOCUMENT_2                -    0.00463487214184523
FLAG_DOCUMENT_4                -    0.0028446737728748356
FLAG_DOCUMENT_5                -    0.00018901328908299872
FLAG_DOCUMENT_7                -    0.0004998065647304965
FLAG_DOCUMENT_9                -    0.002624602568561614
FLAG_DOCUMENT_10               -    0.0013798208494406347
FLAG_DOCUMENT_11               -    0.0031357732194646874
FLAG_DOCUMENT_17               -    0.00367027459434038
FLAG_DOCUMENT_19              

Let us see if we have any missing values present in our dataset.

In [11]:
missing_data_info = df.isna().sum()
missing_data_info = missing_data_info[missing_data_info.gt(0)]
print(f"There are {len(missing_data_info)} columns with missing values.\n")

missing_data_info_sorted = dict(sorted(missing_data_info.items(), key=lambda item: item[1]))
for column_name, num_of_null_values in missing_data_info_sorted.items():
    print("{: <30} -    {:<8}".format(column_name, num_of_null_values))

There are 60 columns with missing values.

CNT_FAM_MEMBERS                -    1       
DAYS_LAST_PHONE_CHANGE         -    1       
AMT_ANNUITY                    -    6       
AMT_GOODS_PRICE                -    167     
EXT_SOURCE_2                   -    413     
OBS_30_CNT_SOCIAL_CIRCLE       -    639     
DEF_30_CNT_SOCIAL_CIRCLE       -    639     
OBS_60_CNT_SOCIAL_CIRCLE       -    639     
DEF_60_CNT_SOCIAL_CIRCLE       -    639     
NAME_TYPE_SUITE                -    770     
AMT_REQ_CREDIT_BUREAU_MON      -    24998   
AMT_REQ_CREDIT_BUREAU_YEAR     -    24998   
EXT_SOURCE_3                   -    36656   
OCCUPATION_TYPE                -    57867   
EMERGENCYSTATE_MODE            -    87336   
TOTALAREA_MODE                 -    88948   
YEARS_BEGINEXPLUATATION_AVG    -    89900   
YEARS_BEGINEXPLUATATION_MODE   -    89900   
YEARS_BEGINEXPLUATATION_MEDI   -    89900   
FLOORSMAX_AVG                  -    91726   
FLOORSMAX_MODE                 -    91726   
FLOORSMAX_ME

If we observe the number of missing values in following columns:
1. CNT_FAM_MEMBERS
2. DAYS_LAST_PHONE_CHANGE
3. AMT_ANNUITY
4. AMT_GOODS_PRICE
5. EXT_SOURCE_2
6. OBS_30_CNT_SOCIAL_CIRCLE
7. DEF_30_CNT_SOCIAL_CIRCLE
8. OBS_60_CNT_SOCIAL_CIRCLE
9. DEF_60_CNT_SOCIAL_CIRCLE
10. NAME_TYPE_SUITE  
  
We can see that the number of missing values is relatively very small as compared to the total number of data points, so we can remove the datapoints that contain these missing values.

In [12]:
datapoints_before = len(df)
print(f"Number of datapoints before  : {datapoints_before}")
df.dropna(
    subset=[
        "CNT_FAM_MEMBERS", 
        "DAYS_LAST_PHONE_CHANGE",
        "AMT_ANNUITY",
        "AMT_GOODS_PRICE",
        "EXT_SOURCE_2",
        "OBS_30_CNT_SOCIAL_CIRCLE",
        "DEF_30_CNT_SOCIAL_CIRCLE",
        "OBS_60_CNT_SOCIAL_CIRCLE",
        "DEF_60_CNT_SOCIAL_CIRCLE",
        "NAME_TYPE_SUITE",
    ],
    how="any",
    inplace=True,
)
print(f"Number of datapoints after   : {len(df)}")
print(f"Total number of rows removed : {datapoints_before - len(df)}")

Number of datapoints before  : 184506
Number of datapoints after   : 182683
Total number of rows removed : 1823


A credit inquiry is a request put up by a financial institution or firm for the credit report information of an individual or entity from a credit reporting bureau. It is of two types -  
  
**1. Hard inquiries:** These are typically inquiries by lenders after you apply for the credit. These inquiries will impact your credit score because most credit score models look at how recently and how frequently you apply for credit.   
  
**2. Soft inquiries:** These are reviews of your credit file, including reviews of existing accounts by lenders, prescreenig inquiries by lenders, and your request for your annual credit report. These will not change your credit score.  
  
     
The following columns contain the information about the credit enquiries:
1. AMT_REQ_CREDIT_BUREAU_HOUR
2. AMT_REQ_CREDIT_BUREAU_DAY
3. AMT_REQ_CREDIT_BUREAU_WEEK
4. AMT_REQ_CREDIT_BUREAU_MON
5. AMT_REQ_CREDIT_BUREAU_QRT
6. AMT_REQ_CREDIT_BUREAU_YEAR  

Each of these columns contain `24,998` missing values.
To deal with the missing values problem, let's replace the missing values with the mode of each column i.e. the most occurring value in the column.  
Here mode is calculated by excluding the missing values.

In [13]:
credit_enquiries_columns = [
    "AMT_REQ_CREDIT_BUREAU_HOUR",
    "AMT_REQ_CREDIT_BUREAU_DAY",
    "AMT_REQ_CREDIT_BUREAU_WEEK",
    "AMT_REQ_CREDIT_BUREAU_MON",
    "AMT_REQ_CREDIT_BUREAU_QRT",
    "AMT_REQ_CREDIT_BUREAU_YEAR"
]

for credit_enquiry_column in credit_enquiries_columns:
    try:
        mode_of_credit_enquiry_column = df[df[credit_enquiry_column].notnull()][credit_enquiry_column].mode()[0]
        df[credit_enquiry_column].fillna(
            value=mode_of_credit_enquiry_column,
            inplace=True
        )
        print("Column: {: <30}     Mode: {:<8}".format(credit_enquiry_column, mode_of_credit_enquiry_column))
    except KeyError as e:
        print(f"{credit_enquiry_column} already dropped.")

AMT_REQ_CREDIT_BUREAU_HOUR already dropped.
AMT_REQ_CREDIT_BUREAU_DAY already dropped.
AMT_REQ_CREDIT_BUREAU_WEEK already dropped.
Column: AMT_REQ_CREDIT_BUREAU_MON          Mode: 0.0     
AMT_REQ_CREDIT_BUREAU_QRT already dropped.
Column: AMT_REQ_CREDIT_BUREAU_YEAR         Mode: 0.0     


The columns `EXT_SOURCE_1` and `EXT_SOURCE_3` are the normalized scores from external data source and are important. But they have a significant number of missing values.   
To deal with missing values let's replace the missing values with the mean of the column.

In [14]:
external_source_score_columns = [
    "EXT_SOURCE_1",
    "EXT_SOURCE_3",
]

for external_source_score_column in external_source_score_columns:
    mean_of_external_source_score_column = df[df[external_source_score_column].notnull()][external_source_score_column].mean()
    df[external_source_score_column].fillna(
        value=mean_of_external_source_score_column,
        inplace=True
    )
    print("Column: {: <10}     Mean: {:<8}".format(external_source_score_column, mean_of_external_source_score_column))

Column: EXT_SOURCE_1     Mean: 0.5023622654099332
Column: EXT_SOURCE_3     Mean: 0.5107575584647581


Since the number of missing values in column `OCCUPATION_TYPE` is quite large, let's replace the missing values with another categoru called `Not available`.

In [15]:
df["OCCUPATION_TYPE"].fillna("Not available",inplace=True)

Now, the rest of the columns with missing values have a high proportion of values missing. So to build a baseline model, let's just drop all those features.

In [16]:
missing_data_info = df.isna().sum()
missing_data_info = missing_data_info[missing_data_info.gt(0)]
for column_name, num_of_null_values in missing_data_info_sorted.items():
    print(f"{column_name}")
    df.drop(
    columns=[column_name],
    inplace=True,
        axis=1,
    )

CNT_FAM_MEMBERS
DAYS_LAST_PHONE_CHANGE
AMT_ANNUITY
AMT_GOODS_PRICE
EXT_SOURCE_2
OBS_30_CNT_SOCIAL_CIRCLE
DEF_30_CNT_SOCIAL_CIRCLE
OBS_60_CNT_SOCIAL_CIRCLE
DEF_60_CNT_SOCIAL_CIRCLE
NAME_TYPE_SUITE
AMT_REQ_CREDIT_BUREAU_MON
AMT_REQ_CREDIT_BUREAU_YEAR
EXT_SOURCE_3
OCCUPATION_TYPE
EMERGENCYSTATE_MODE
TOTALAREA_MODE
YEARS_BEGINEXPLUATATION_AVG
YEARS_BEGINEXPLUATATION_MODE
YEARS_BEGINEXPLUATATION_MEDI
FLOORSMAX_AVG
FLOORSMAX_MODE
FLOORSMAX_MEDI
HOUSETYPE_MODE
LIVINGAREA_AVG
LIVINGAREA_MODE
LIVINGAREA_MEDI
ENTRANCES_AVG
ENTRANCES_MODE
ENTRANCES_MEDI
APARTMENTS_AVG
APARTMENTS_MODE
APARTMENTS_MEDI
WALLSMATERIAL_MODE
ELEVATORS_AVG
ELEVATORS_MODE
ELEVATORS_MEDI
NONLIVINGAREA_AVG
NONLIVINGAREA_MODE
NONLIVINGAREA_MEDI
EXT_SOURCE_1
BASEMENTAREA_AVG
BASEMENTAREA_MODE
BASEMENTAREA_MEDI
LANDAREA_AVG
LANDAREA_MODE
LANDAREA_MEDI
OWN_CAR_AGE
YEARS_BUILD_AVG
YEARS_BUILD_MODE
YEARS_BUILD_MEDI
FLOORSMIN_AVG
FLOORSMIN_MODE
FLOORSMIN_MEDI
LIVINGAPARTMENTS_AVG
LIVINGAPARTMENTS_MODE
LIVINGAPARTMENTS_MEDI
FONDKAP

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182683 entries, 0 to 184505
Data columns (total 36 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   NAME_CONTRACT_TYPE           182683 non-null  object 
 1   CODE_GENDER                  182683 non-null  object 
 2   FLAG_OWN_CAR                 182683 non-null  object 
 3   FLAG_OWN_REALTY              182683 non-null  object 
 4   CNT_CHILDREN                 182683 non-null  int64  
 5   AMT_CREDIT                   182683 non-null  float64
 6   NAME_INCOME_TYPE             182683 non-null  object 
 7   NAME_EDUCATION_TYPE          182683 non-null  object 
 8   NAME_FAMILY_STATUS           182683 non-null  object 
 9   NAME_HOUSING_TYPE            182683 non-null  object 
 10  REGION_POPULATION_RELATIVE   182683 non-null  float64
 11  DAYS_BIRTH                   182683 non-null  int64  
 12  DAYS_EMPLOYED                182683 non-null  int64  
 13 

In [18]:
categorical_columns = [
    "NAME_CONTRACT_TYPE",
    "CODE_GENDER",
    "FLAG_OWN_CAR",
    "FLAG_OWN_REALTY",
    "NAME_INCOME_TYPE",
    "NAME_EDUCATION_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_HOUSING_TYPE",
    "WEEKDAY_APPR_PROCESS_START",
    "ORGANIZATION_TYPE",
]

In [19]:
for column in categorical_columns:
    print(column)
    categories = df[column].unique()
    print(f"Number of unique values: {len(categories)}")
    for category in categories:
        print(category)
    print("\n\n")

NAME_CONTRACT_TYPE
Number of unique values: 2
Cash loans
Revolving loans



CODE_GENDER
Number of unique values: 3
F
M
XNA



FLAG_OWN_CAR
Number of unique values: 2
N
Y



FLAG_OWN_REALTY
Number of unique values: 2
Y
N



NAME_INCOME_TYPE
Number of unique values: 8
Commercial associate
Working
Pensioner
State servant
Student
Businessman
Unemployed
Maternity leave



NAME_EDUCATION_TYPE
Number of unique values: 5
Secondary / secondary special
Higher education
Incomplete higher
Lower secondary
Academic degree



NAME_FAMILY_STATUS
Number of unique values: 5
Married
Civil marriage
Single / not married
Separated
Widow



NAME_HOUSING_TYPE
Number of unique values: 6
House / apartment
With parents
Municipal apartment
Office apartment
Rented apartment
Co-op apartment



WEEKDAY_APPR_PROCESS_START
Number of unique values: 7
WEDNESDAY
TUESDAY
MONDAY
SUNDAY
SATURDAY
FRIDAY
THURSDAY



ORGANIZATION_TYPE
Number of unique values: 58
Business Entity Type 3
Transport: type 4
Other
Self-employed
XNA


In [20]:
column_names_to_one_hot = categorical_columns

print(f"Shape of data before: {df.shape}")
df = pd.get_dummies(df, columns=column_names_to_one_hot)
print(f"Shape of data after: {df.shape}")

Shape of data before: (182683, 36)
Shape of data after: (182683, 124)


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182683 entries, 0 to 184505
Columns: 124 entries, CNT_CHILDREN to ORGANIZATION_TYPE_XNA
dtypes: float64(3), int64(23), uint8(98)
memory usage: 54.7 MB


In [22]:
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values

In [23]:
clf = LogisticRegression(random_state=0).fit(X, Y)

In [24]:
test_data = pd.read_csv(os.path.join(DATA_DIR, "test_data.csv"))

In [25]:
test_data.shape

(123005, 121)

In [26]:
test_data.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,367294,Cash loans,F,N,Y,0,180000.0,265306.5,25317.0,252000.0,...,0,0,0,0,0.0,0.0,0.0,1.0,2.0,5.0
1,439847,Cash loans,F,N,Y,0,202500.0,346500.0,21069.0,346500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0
2,380562,Cash loans,M,Y,N,0,360000.0,545040.0,36553.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,5.0
3,407238,Cash loans,F,N,Y,0,135000.0,307557.0,20682.0,265500.0,...,0,0,0,0,0.0,0.0,1.0,0.0,1.0,2.0
4,239910,Cash loans,F,N,Y,0,157500.0,1056447.0,31018.5,922500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# columns_to_drop = [
#     "AMT_INCOME_TOTAL",           
#     "FLAG_CONT_MOBILE",           
#     "FLAG_EMAIL",                 
#     "REG_REGION_NOT_LIVE_REGION", 
#     "LIVE_REGION_NOT_WORK_REGION",
#     "NONLIVINGAPARTMENTS_AVG",    
#     "NONLIVINGAPARTMENTS_MODE",  
#     "NONLIVINGAPARTMENTS_MEDI",  
#     "FLAG_DOCUMENT_2",           
#     "FLAG_DOCUMENT_4",           
#     "FLAG_DOCUMENT_5",           
#     "FLAG_DOCUMENT_7",           
#     "FLAG_DOCUMENT_9",           
#     "FLAG_DOCUMENT_10",          
#     "FLAG_DOCUMENT_11",          
#     "FLAG_DOCUMENT_17",          
#     "FLAG_DOCUMENT_19",          
#     "FLAG_DOCUMENT_20",          
#     "FLAG_DOCUMENT_21",          
#     "AMT_REQ_CREDIT_BUREAU_HOUR",
#     "AMT_REQ_CREDIT_BUREAU_DAY", 
#     "AMT_REQ_CREDIT_BUREAU_WEEK",
#     "AMT_REQ_CREDIT_BUREAU_QRT",
# ]

# test_data.drop(
#     columns=columns_to_drop,
#     axis=1,
#     inplace=True,
# )